Linear  Regression

Adapt the datas 

In [19]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
import numpy as np

In [20]:
df = pd.read_csv("Start-Up_final.csv")

In [21]:
df.drop(columns="Unnamed: 0", inplace = True)

df_final = df

In [22]:
# Instanciation de l'encodeur
encoder = LabelEncoder()  # sparse=False pour obtenir un DataFrame dense

# Encodage
encoded_array = encoder.fit_transform(df[['status']])

# Transformation en DataFrame
encoded_df = pd.DataFrame(encoded_array, columns=["status"])

# Fusionner avec le DataFrame initial et supprimer l'ancienne colonne
df_final = pd.concat([df.drop(columns=['status']), encoded_df], axis=1)


c:\Users\theof\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\preprocessing\_label.py:110: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [23]:
df_final

,category_list,funding_total_usd,country_code,funding_rounds,founded_at,first_funding_at,last_funding_at,status
0,Media,10000000,IND,1,NaN,2015-01-05,2015-01-05,3
1,Social Network Media,700000,USA,2,2014-09-04,2014-03-01,2014-10-14,3
2,Mobile,3406878,NaN,1,NaN,2014-01-30,2014-01-30,3
3,Curated Web,2000000,CHN,1,2007-01-01,2008-03-19,2008-03-19,3
4,Software,-,USA,1,2010-01-01,2014-07-24,2014-07-24,3
...,...,...,...,...,...,...,...,...
66363,Enterprise Software,1587301,CHN,1,NaN,2012-04-01,2012-04-01,3
66364,Mobile,114304,HRV,4,2012-05-13,2011-11-01,2014-03-01,3
66365,unknown,-,NaN,1,2011-01-01,2014-08-01,2014-08-01,3
66366,Telecommunications,18192,USA,1,2014-01-01,2015-01-01,2015-01-01,3


Date

In [24]:
print(df_final["founded_at"].value_counts().head(1))
print(df_final["first_funding_at"].value_counts().head(1))
print(df_final["last_funding_at"].value_counts().head(1))

founded_at
2012-01-01    2730
Name: count, dtype: int64
first_funding_at
2014-01-01    565
Name: count, dtype: int64
last_funding_at
2014-01-01    483
Name: count, dtype: int64


In [25]:
df_final["last_funding_at"] = pd.to_datetime(df_final["last_funding_at"], format ="%Y-%m-%d" , errors ="coerce")

df_final["last_funding_at"] = df_final["last_funding_at"].dt.year.astype("Int64")

df_final["first_funding_at"] = pd.to_datetime(df_final["first_funding_at"], format ="%Y-%m-%d", errors = "coerce")

df_final["first_funding_at"] = df_final["first_funding_at"].dt.year.astype("Int64")

df_final["founded_at"] = pd.to_datetime(df_final["founded_at"], format ="%Y-%m-%d", errors = "coerce")

df_final["founded_at"] = df_final["founded_at"].dt.year.astype("Int64")

df_final["last_funding_at"] = df_final["last_funding_at"].fillna(2015).astype("Int64")
df_final["first_funding_at"] = df_final["first_funding_at"].fillna(2014).astype("Int64")
df_final["founded_at"] = df_final["founded_at"].fillna(2012).astype("Int64")

Countries

We merge together the countries that had extremly small start-up (10 or less) in order to reduce noise and the possibility of overfitting

In [26]:
threshold = 10  # Seuil pour définir un pays rare
country_counts = df['country_code'].value_counts()

df_final['country_code'] = df_final['country_code'].apply(lambda x: x if country_counts.get(x, 0) > threshold else 'Other')


In [27]:
label_encoder = LabelEncoder()

df_final["country_code"] = label_encoder.fit_transform(df_final["country_code"])


Category_list

In [28]:
df_final["category_list"] = label_encoder.fit_transform(df_final["category_list"])

In [29]:
df_final

,category_list,funding_total_usd,country_code,funding_rounds,founded_at,first_funding_at,last_funding_at,status
0,32,10000000,30,1,2012,2015,2015,3
1,55,700000,68,2,2014,2014,2014,3
2,36,3406878,49,1,2012,2014,2014,3
3,12,2000000,11,1,2007,2008,2008,3
4,56,-,68,1,2010,2014,2014,3
...,...,...,...,...,...,...,...,...
66363,18,1587301,11,1,2012,2012,2012,3
66364,36,114304,49,4,2012,2011,2014,3
66365,65,-,49,1,2011,2014,2014,3
66366,60,18192,68,1,2014,2015,2015,3


In [30]:
df_final["funding_total_usd"] = pd.to_numeric(df_final["funding_total_usd"], errors="coerce")

df_final.dropna(subset=['funding_total_usd'], inplace=True)



Logistic regression

In [39]:
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler



X = df_final.drop(columns=["status"])  # Features
y = df_final["status"]  # Cible

# Split train-test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Normalisation des features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)  # Fit + transform sur train
X_test_scaled = scaler.transform(X_test)  # Seulement transform sur test (sans refit)

# 🔹 Correction : Rétablir le DataFrame avec les bons index
X_train_scaled = pd.DataFrame(X_train_scaled, columns=X.columns, index=X_train.index)
X_test_scaled = pd.DataFrame(X_test_scaled, columns=X.columns, index=X_test.index)

# Création du modèle et entraînement
model = LogisticRegression(solver="lbfgs", max_iter=10000)
model.fit(X_train_scaled, y_train)

# Prédictions
y_pred = model.predict(X_test_scaled)




In [40]:

# Calcul des métriques
r2 = r2_score(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
rmse = mse ** 0.5  # Racine carrée du MSE

print(f"R² Score : {r2:.2f}")
print(f"MSE : {mse:.2f}")
print(f"RMSE : {rmse:.2f}")

R² Score : -0.21
MSE : 1.17
RMSE : 1.08
